A notebook to use Ian Sullivans StarFast to simulate images at different stellar densities 

In [4]:
# necessary : tells python where to look for StarFast...
import sys
starfast_home = '/Users/chris/starfast_simulator/'
sys.path.insert(0, starfast_home)


In [11]:
import galsim 
from lsst.sims.photUtils import matchStar
import numpy as np
from StarFast import StarSim
from matplotlib import pyplot as plt
import os
import imp
import pickle
from lsst.afw.geom import Angle
from lsst.sims.photUtils import matchStar
import _pickle as cPickle  

In [5]:
imp.load_source('calc_refractive_index',starfast_home+'calc_refractive_index.py')
imp.load_source('fast_dft', starfast_home+'fast_dft.py')
imp.load_source('StarFast', starfast_home+'StarFast.py')

<module 'StarFast' from '/Users/chris/starfast_simulator/StarFast.py'>

In [6]:
# setup the observatory location
lsst_lat = Angle(np.radians(-30.244639))
lsst_lon = Angle(np.radians(-70.749417))
lsst_alt = 2663.

In [40]:
# setup the general parameters to use for all of the simulations

seed = 5  # Seed for the random number generator. Simulations with the same seed are reproduceable
dimension = 1024  # Number of pixels on a side for the simulated image
#n_star = 10000  # Number of stars to model in the simulated catalog.
# The catalog covers an area ~4x larger than the area, to allow simulated 
# rotations and dithering observations
pixel_scale = 0.25  # plate scale, in arcseconds/pixel
psf_fwhm = 0.25  # FWHM of the PSF, in arcseconds



In [36]:
# Load the density definitions from MAF lt 24.5,   southern sky only ....
density_dic = np.load('../data_products/MAF_density_definitions/density_r_lt_245_S.npy').item()

# translate to needed 
# number of stars per area of 
# dimension  and pixel scale as defined above 
# for densitiy levels corresponding to 
# keys of density_dic 

area_in_sq_deg = 4*(dimension**2.0 ) * (pixel_scale**2.0)  / (3600 * 3600)
N_stars_dic = {} 
for key in dic.keys():
    N_stars_dic[key] = area_in_sq_deg * density_dic[key]

{0.01: 2587041.5847659572,
 0.05: 710251.18927021686,
 0.1: 305078.39864074055,
 0.15: 170992.79863542572,
 0.2: 110116.79435121812,
 0.25: 76499.999999999927}

In [44]:
#N_stars_dic

In [62]:
## Path to the directory to save output
output_directory = "../data_products/StarFast_simulations/test%1i/" % seed  
    

if not os.path.exists(output_directory): os.system('mkdir %s' % output_directory)
print('We will save the simulations in  %s'%output_directory)

and_files_directory = output_directory+"and_files/"
if not os.path.exists(and_files_directory): os.system('mkdir %s' % and_files_directory)
    
images_directory =     output_directory + "images/"
if not os.path.exists(images_directory): os.system('mkdir %s' % images_directory)



We will save the simulations in  ../data_products/StarFast_simulations/test5/


In [9]:



exposureId = 0  # Unique exposure identification number. Also used as the "OBSID"
elevation_min = 30.0  # Minimum observation elevation angle to simulate, in degrees
elevation_max = 90.0  # Open maximum observation angle, in degrees. 
# Only angles less than elevation_max will be simulated
elevation_step = 5  # Elevation angle step size, in degrees.


# Not likely to want to change these

hottest_star = 'B'  # Hottest star to include (types are 'OBAFGKMR')
coolest_star = 'M'  # Coolest star to include
wavelength_step = 10  # Wavelength resolution of the spectra and calculation of filter and DCR effects. In nm.
ra_offset = Angle(0)  # Additional offset in RA from the field center, for dithering. In radians as an LSST Angle object
dec_offset = Angle(0)  # Additional offset in Dec from the field center, for dithering. In radians as an LSST Angle object
sky_rotation = 0.0  # Sky rotation angle, in Degrees. I realize this is different than RA and Dec
instrument_noise = 0.  # Adds noise akin to instrumental noise (post-PSF). Set to 1.0 for default value, can be scaled up or down
photon_noise = 1./15.  # Adds poisson noise akin to photon shot noise. Set to 1.0 for default value, can be scaled up or down
sky_noise = 0  # Adds noise prior to convolving with the PSF.
band_dict = {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}  # LSST filter numbers used by the butler


attenuation = 20. # attenuation factor that was used in the simulations

ra = lsst_lon + ra_offset
dec = lsst_lat + dec_offset

In [16]:
# Load simulated Kurucz stellar SEDs. Cache the list for later use. 

pickle_file = 'sed_list.pickle'
if os.path.exists(pickle_file):
    with open(pickle_file, 'rb') as dumpfile:
        sed_list = pickle.load(dumpfile)
else:
    matchStarObj = matchStar()
    sed_list = matchStarObj.loadKuruczSEDs()
    with open(pickle_file, 'wb') as dumpfile:
        pickle.dump(sed_list, dumpfile, pickle.HIGHEST_PROTOCOL)


# Set up the PSF. 
gsp = galsim.GSParams(folding_threshold=1.0 / (dimension), maximum_fft_size=12288)
psf = galsim.Kolmogorov(fwhm=psf_fwhm / pixel_scale, flux=1, gsparams=gsp)



In [17]:
# u-band simulation 
band_name = 'r'

# initialize the simulation class ... 
sim = StarSim(psf=psf, pixel_scale=pixel_scale, x_size=dimension, y_size=dimension,
              band_name=band_name, wavelength_step=wavelength_step,
              sed_list=sed_list, ra=ra, dec=dec, sky_rotation=sky_rotation, 
              attenuation=attenuation)




In [78]:
N_stars_dic

{0.01: 52328.505339265983,
 0.05: 14366.364796300288,
 0.1: 6170.8697325060411,
 0.15: 3458.6987812874258,
 0.2: 2227.350072407849,
 0.25: 1547.3777777777764}

# Loop over densities : set of images for different densities ... 

In [76]:
from lsst.sims.photUtils import Bandpass, matchStar, PhotometricParameters

In [77]:
# First consider moderate density region
for density in N_stars_dic.keys():
    
    n_star = int(N_stars_dic[density])
    print('\nSetting n_star=%d'%n_star)
    print('For top %s percent density regime'%str(density))

    '''
    Simulate a catalog of stars, with fluxes and SEDs
    '''
    sim.load_catalog(n_star=n_star, hottest_star=hottest_star, 
                     coolest_star=coolest_star, seed=seed)

    '''
    Write the catalog to disk. Note that this catalog includes 
    fluxes for all 6 LSST bands if filter_list is None
    ''' 

    # note : magnitude limit is to upper limit, 
    # in band set by band_name above ... 
    sim.make_reference_catalog(output_directory=output_directory + "and_files/", 
                               filter_list=None, magnitude_limit=24.5)

    '''
    Generate the raw simulation
    '''
    sim.simulate()


    # what does this step do ?
    # I assume it is making the exposure... 
    # since I am using band 'r', 
    # I would also set the bandpass to 'r'  , rather than 'g' ... 

    photo = PhotometricParameters(exptime=30., nexp=1, platescale=0.25, bandpass='r')

    expId = exposureId + 100*band_dict[band_name]

    # I don't need to change these since I'm not simulating 
    # varying airmass ... 
    elevation = 90
    azimuth = 0 

    exposure = sim.convolve(elevation=elevation, azimuth=azimuth,
                            instrument_noise=instrument_noise, sky_noise=sky_noise,
                            photon_noise=photon_noise, exposureId=expId, obsid=expId)
    filename = "lsst_e_%3.3i_f%i_d%s.fits" % (expId, band_dict[band_name],str(density))
    expId += 1
    exposure.writeFits(output_directory + "images/" + filename)



Setting n_star=52328
For top 01 percent density regime
Number and flux contribution of stars of each type:
 [M 39999| 1.48%] [K 6384| 1.81%] [G 3990| 3.38%] [F 1585| 4.01%] [A 314| 12.56%] [B 56| 76.76%] [O 0| 0.00%]
Writing 52328 stars brighter than 24.5 mag to reference catalog in 6 bands
Min/max magnitude:  3.80841594488 20.8673488623
Simulating 13142 stars within observable region
Time to model 13129 stars: [19.150s | 0.00146s per star]
Time to model 13 bright stars: [3.909s | 0.30067s per star]
FFT timing for 8 DCR planes: [1.984s | 0.248s per plane]
FFT timing for 8 DCR planes: [8.296s | 1.037s per plane]
Setting n_star=14366
For top 05 percent density regime
Number and flux contribution of stars of each type:
 [M 10966| 1.87%] [K 1751| 2.19%] [G 1101| 3.94%] [F 448| 5.30%] [A 82| 13.62%] [B 18| 73.08%] [O 0| 0.00%]
Writing 14366 stars brighter than 24.5 mag to reference catalog in 6 bands
Min/max magnitude:  5.01942118091 20.8739751822
Simulating 3587 stars within observable reg

In [81]:
for density in N_stars_dic.keys():
    
    n_star = int(N_stars_dic[density])
    print('\nSetting n_star=%d'%n_star)
    print('For top %s percent density regime'%str(density))


Setting n_star=52328
For top 0.01 percent density regime

Setting n_star=14366
For top 0.05 percent density regime

Setting n_star=6170
For top 0.1 percent density regime

Setting n_star=3458
For top 0.15 percent density regime

Setting n_star=2227
For top 0.2 percent density regime

Setting n_star=1547
For top 0.25 percent density regime


90.0